In [1]:
import numpy as np

Three jugs $A$, $B$, and $C$ have volumes $V_A=\sqrt{5}$ , $V_B=\sqrt{3}$ , and $V_C=\sqrt{2}$ liters, respectively. There is a tap from which we can fill the jugs, and a sink into which we can empty them. At the beginning of the problem, the three jugs are empty.

To achieve the solution of the problem, we perform a sequence of steps that can only be one of the following:

Pour water from the tap into a jug until it is filled to the top.
Empty a jug into the sink.
Pour water from one jug into another until either the source jug is empty or the receiving jug is filled to the top, whichever happens first.
We assume that these steps are performed with absolute accuracy and no water is spilled.

Our goal is to measure a specific amount of water, meaning we seek to arrive at a state where any one of the jugs (no matter which one) contains this specific amount. As it is impossible to measure with these jugs any rational, non-zero volume with a finite number of steps, we will relax the condition: Instead of measuring a volume of 1 , we measure a volume of $1 \pm 0.0003$ liters, i.e., any volume in the interval $[0.9997, 1.0003]$ will suffice.

The solution should be presented as a list of two-letter strings that represent the steps of the solution, in which the first letter denotes the source, and the second letter denotes the recipient. The tap is denoted by $T$ and the sink by $S$. The list should also be preceded by a number representing the minimum number of steps.

For example: If the task is to measure $1 \pm 0.01$ liters, the fastest solution would require six steps and could be described as
6 TA AB BS AB TA AB
Upon completion of the above sequence of six steps, jug A would contain 1.00803 liters of water, a volume within the specified tolerance.

Your goal: Find a sequence of steps measuring $1 \pm 0.0003$ liters such that the number of steps is minimal. Submit your answer using the above format.

Lemma 1
===========

Each jug, at any time stem, contains an integer-weighted (positive and negative) sum of $V_A, V_B, V_C$

Proof
-----

Consider a pour from $A$ to $B$.  Before the pour, $A$ contains $a_1 V_A + b_1 V_B + c_1 V_C$ where $a_1, b_1, c_1$ are (possibly negative) integers, while $B$ contains $a_2 V_A + b_2 V_B + c_2 V_C$ where $a_2, b_2, c_2$ are also integers.

After the pour, $A$ contains $\max( (a_1 + a_2) V_A + (b_1 + b_2 - 1) V_B + (c_1 + c_2) V_C, 0)$, wihle $B$ contains $\min(V_B, (a_1 + a_2) V_A + (b_1+b_2) V_B + (c_1 + c_2)V_C)$.  Each of these are integer-weighted sums of $V_A$, $V_B$, and $V_C$

In [2]:
def is_valid(n_a, n_b, n_c):
    val = n_a * np.sqrt(5) + n_b * np.sqrt(3) + n_c * np.sqrt(2)
    return np.abs(val - 1.0) <= 3.0e-4

In [3]:
valids = []

for n_a in range(-10, 10):
    for n_b in range(-10, 10):
        for n_c in range(-10, 10):
            if is_valid(n_a, n_b, n_c):
                valids.append((n_a, n_b, n_c))

In [4]:
len(valids)

0

In [5]:
valids = []

range_size = 20

for n_a in range(-range_size, range_size):
    for n_b in range(-range_size, range_size):
        for n_c in range(-range_size, range_size):
            if is_valid(n_a, n_b, n_c):
                valids.append((n_a, n_b, n_c))

In [6]:
len(valids)

1

In [7]:
valids

[(-7, -1, 13)]

In [8]:
7+1+13

21

In [9]:
more_valids = []
range_bound = 40

for n_a in range(-range_bound, range_bound+1):
    remaining = range_bound - n_a
    for n_b in range(-remaining, remaining+1):
        still_left = range_bound - n_a - n_b
        for n_c in range(-still_left, still_left+1):
            if is_valid(n_a, n_b, n_c):
                more_valids.append((n_a, n_b, n_c))
                print(f"Found one : ${(n_a, n_b, n_c)}")

Found one : $(-7, -1, 13)
Found one : $(0, -28, 35)


We found one!
-------

Now we need to find a path to get to it.

We'll start by writing a simulator

In [10]:
def volume_of(weights):
    return np.dot(weights, np.sqrt([5,3,2]))

def pour(source, dest, weights):
    new_d = weights[dest, :] + weights[source, :]
    alt_d = np.zeros_like(new_d)
    alt_d[dest] = 1
    if volume_of(new_d) > volume_of(alt_d):
        new_s = new_d - alt_d
        result = np.copy(weights)
        result[source, :] = new_s
        result[dest, :] = alt_d
        return result
    result = np.copy(weights)
    result[source, :] = np.zeros_like(new_d)
    result[dest, :] = new_d
    return result

def to_sink(source, weights):
    result = np.copy(weights)
    result[source, :] = np.zeros_like(result[source, :])
    return result

def from_source(dest, weights):
    result = np.copy(weights)
    new_d = np.zeros_like(result[dest, :])
    new_d[dest] = 1
    result[dest, :] = new_d
    return result

In [11]:
pour(1, 0, from_source(1, np.zeros((3,3), dtype='int')))

array([[0, 1, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [12]:
state = pour(1, 0, from_source(1, np.zeros((3,3), dtype='int')))
state = from_source(1, state)
state, pour(1,0,state), pour(0,2, state)

(array([[0, 1, 0],
        [0, 1, 0],
        [0, 0, 0]]),
 array([[ 1,  0,  0],
        [-1,  2,  0],
        [ 0,  0,  0]]),
 array([[ 0,  1, -1],
        [ 0,  1,  0],
        [ 0,  0,  1]]))

In [13]:
class PourState:
    def __init__(self):
        self.state = np.zeros((3,3), dtype='int')
        self.history = []
    def do_pour(self, source, dest):
        self.state = pour(source, dest, self.state)
        self.history.append("ABC"[source] + "ABC"[dest])
    def do_fill(self, dest):
        self.state = from_source(dest, self.state)
        self.history.append("T" + "ABC"[dest])
    def do_dump(self, source):
        self.state = to_sink(source, self.state)
        self.history.append("ABC"[source] + "S")
    def clone_self(self):
        result = PourState()
        result.state = np.copy(self.state)
        result.history= [c for c in self.history]
        return result
    def search_heuristic(self, goal):
        g = np.array(goal).reshape((1,3))
        deltas = np.abs(g - self.state)
        deltas = np.sum(deltas, axis=1)
        return np.min(deltas)

In [14]:
more_valids

[(-7, -1, 13), (0, -28, 35)]

In [15]:
ps = PourState()
ps.do_fill(2)
ps.state

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 1]])

In [16]:
ps.do_pour(2,1)
ps.state

array([[0, 0, 0],
       [0, 0, 1],
       [0, 0, 0]])

In [17]:
ps.do_fill(2)
ps.do_pour(2,1)
ps.state

array([[ 0,  0,  0],
       [ 0,  1,  0],
       [ 0, -1,  2]])

In [18]:
ps.do_pour(2,0)
ps.state

array([[ 0, -1,  2],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [19]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-1, -1,  3]])

In [20]:
ps.do_dump(0)
ps.do_pour(2,0)
ps.state

array([[-1, -1,  3],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [21]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[-1, -1,  4],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [22]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-2, -1,  5]])

In [23]:
ps.do_dump(0)
ps.state

array([[ 0,  0,  0],
       [ 0,  1,  0],
       [-2, -1,  5]])

In [24]:
ps.do_pour(2,0)
ps.state

array([[-2, -1,  5],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [25]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-3, -1,  6]])

In [26]:
ps.do_dump(0)
ps.do_pour(2,0)
ps.state

array([[-3, -1,  6],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [27]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[-3, -1,  7],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [28]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-4, -1,  8]])

In [29]:
ps.do_dump(0)
ps.do_pour(2,0)
ps.state

array([[-4, -1,  8],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [30]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[-4, -1,  9],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [31]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-5, -1, 10]])

In [32]:
ps.do_dump(0)
ps.do_pour(2,0)
ps.state

array([[-5, -1, 10],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [33]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-6, -1, 11]])

In [34]:
ps.do_dump(0)
ps.do_pour(2,0)
ps.state

array([[-6, -1, 11],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [35]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[-6, -1, 12],
       [ 0,  1,  0],
       [ 0,  0,  0]])

In [36]:
ps.do_fill(2)
ps.do_pour(2,0)
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-7, -1, 13]])

In [37]:
ps.history

['TC',
 'CB',
 'TC',
 'CB',
 'CA',
 'TC',
 'CA',
 'AS',
 'CA',
 'TC',
 'CA',
 'TC',
 'CA',
 'AS',
 'CA',
 'TC',
 'CA',
 'AS',
 'CA',
 'TC',
 'CA',
 'TC',
 'CA',
 'AS',
 'CA',
 'TC',
 'CA',
 'TC',
 'CA',
 'AS',
 'CA',
 'TC',
 'CA',
 'AS',
 'CA',
 'TC',
 'CA',
 'TC',
 'CA']

In [38]:
len(ps.history)

39

In [39]:
len([c for c in ps.history if c[0] == 'T']), len([c for c in ps.history if c[1] == 'S'])

(13, 6)

In [40]:
39-13-6

20

In [41]:
even_more_valids = []
range_bound = 40

for n_a in range(-range_bound, range_bound+1):
    remaining = range_bound - abs(n_a)
    for n_b in range(-remaining, remaining+1):
        still_left = range_bound - abs(n_a) - abs(n_b)
        for n_c in range(-still_left, still_left+1):
            if is_valid(n_a, n_b, n_c):
                even_more_valids.append((n_a, n_b, n_c))
                print(f"Found one : ${(n_a, n_b, n_c)}")

Found one : $(-7, -1, 13)


In [42]:
volume_of(ps.state[2,:])

np.float64(1.0002496607828313)

In [43]:
ps2 = PourState()
ps2.state

array([[0, 0, 0],
       [0, 0, 0],
       [0, 0, 0]])

In [44]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[0, 0, 1],
       [0, 0, 0],
       [0, 0, 0]])

In [45]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-1,  0,  2]])

In [46]:
ps2.do_dump(0)
ps2.do_pour(2,0)
ps2.state

array([[-1,  0,  2],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [47]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[-1,  0,  3],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [48]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-2,  0,  4]])

In [49]:
ps2.do_dump(0)
ps2.do_pour(2,0)
ps2.state

array([[-2,  0,  4],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [50]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-3,  0,  5]])

In [51]:
ps2.do_dump(0)
ps2.do_pour(2,0)
ps2.state

array([[-3,  0,  5],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [52]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[-3,  0,  6],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [53]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-4,  0,  7]])

In [54]:
ps2.do_dump(0)
ps2.do_pour(2,0)
ps2.state

array([[-4,  0,  7],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [55]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-5,  0,  8]])

In [56]:
ps2.do_dump(0)
ps2.do_pour(2,0)
ps2.state

array([[-5,  0,  8],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [57]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[-5,  0,  9],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [58]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-6,  0, 10]])

In [59]:
ps2.do_dump(0)
ps2.do_pour(2,0)
ps2.state

array([[-6,  0, 10],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [60]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[-6,  0, 11],
       [ 0,  0,  0],
       [ 0,  0,  0]])

In [61]:
ps2.do_fill(2)
ps2.do_pour(2,0)
ps2.state

array([[ 1,  0,  0],
       [ 0,  0,  0],
       [-7,  0, 12]])

In [62]:
print(ps2.state)
ps2.do_pour(2,1)
ps2.state

[[ 1  0  0]
 [ 0  0  0]
 [-7  0 12]]


array([[ 1,  0,  0],
       [-7,  0, 12],
       [ 0,  0,  0]])

In [63]:
len(ps2.history)

37

In [64]:
ps2.do_fill(2)
ps2.state

array([[ 1,  0,  0],
       [-7,  0, 12],
       [ 0,  0,  1]])

In [65]:
ps2.do_pour(2,1)
ps2.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-7, -1, 13]])

In [66]:
len(ps2.history)

39

In [67]:
len(ps.history)

39

In [68]:
ps.state

array([[ 1,  0,  0],
       [ 0,  1,  0],
       [-7, -1, 13]])

In [69]:
str(len(ps.history)) + " " + " ".join(ps.history)

'39 TC CB TC CB CA TC CA AS CA TC CA TC CA AS CA TC CA AS CA TC CA TC CA AS CA TC CA TC CA AS CA TC CA AS CA TC CA TC CA'

In [70]:
str(len(ps2.history)) + " " + " ".join(ps2.history)

'39 TC CA TC CA AS CA TC CA TC CA AS CA TC CA AS CA TC CA TC CA AS CA TC CA AS CA TC CA TC CA AS CA TC CA TC CA CB TC CB'

In [71]:
len([x for x in ps.history if x == 'TC'])

13

In [72]:
[(x, len([c for c in ps.history if c == x])) for x in set(ps.history)]

[('CA', 18), ('CB', 2), ('TC', 13), ('AS', 6)]

In [73]:
[(x, len([c for c in ps2.history if c == x])) for x in set(ps2.history)]

[('CA', 18), ('CB', 2), ('TC', 13), ('AS', 6)]

Is our feasible solution optimal?
------------------

In [74]:
import heapq

def a_star(goal):
    """And here's the a-star search to find a minimal sequence of moves that gives us our target"""
    initial = PourState()
    queue = []
    explored_count = 0
    depths_seen = set()
    def add_queue(s):
        nonlocal explored_count
        nonlocal depths_seen
        hscore = s.search_heuristic(goal)
        score = len(s.history) + hscore
        heapq.heappush(queue, (score, explored_count, s))
        if not len(s.history) in depths_seen:
            print(f"Adding an item with score {score}, heuristic score {hscore}, and depth {len(s.history)} : q size {len(queue)}")
            depths_seen.add(len(s.history))
        explored_count += 1
    add_queue(initial)
    while len(queue) > 0:
        explore_from = heapq.heappop(queue)[2]
        if explore_from.search_heuristic(goal) == 0:
            return explore_from.clone_self()
        for bucket1 in range(3):
            to_explore = explore_from.clone_self()
            to_explore.do_fill(bucket1)
            add_queue(to_explore)
            to_explore = explore_from.clone_self()
            to_explore.do_dump(bucket1)
            add_queue(to_explore)
            for bucket2 in range(3):
                if bucket2 == bucket1:
                    continue
                to_explore = explore_from.clone_self()
                to_explore.do_pour(bucket1, bucket2)
    return None

Is our feasible solution optimal?
------

Well, a-star search is *not* the right way to find out.  That just uses up all our ram and crashes python!